In [1]:
# Convert DICOMS from RSNA challange to 16 bit PNGS

In [2]:
import os
import multiprocessing

multiprocessing.cpu_count()

4

In [3]:
# pre_resize_dims = None
resize_dim = 1024+512
# apply_voi_lut = False
limit_files = None
files_border_percent=None # 0.5
files_border_process_lower=False,

# None to get cpucount
jobs_count = None

# Constants
U16MAX = 65535

In [4]:
import os
from pathlib import Path
try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except ImportError:
    IS_KAGGLE = False

In [5]:
# if not IS_KAGGLE:
#    !pip install --upgrade kaggle
#    !mkdir ~/.kaggle
#    !mv kaggle.json ~/.kaggle/kaggle.json
#    !kaggle competitions download -c rsna-breast-cancer-detection
#    !unzip -q rsna-breast-cancer-detection.zip -d rsna-breast-cancer-detection
    

In [15]:
competition_path = Path('~/rsna-breast/rsna-breast-cancer-detection').expanduser()

In [7]:
if IS_KAGGLE:
    # !pip uninstall numpy
    !pip install --upgrade --force numpy pandas pydicom dicomsdl tqdm opencv-python pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg python-gdcm pycocotools
else:
    !pip install --upgrade numpy pandas pydicom dicomsdl tqdm opencv-python pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg python-gdcm pycocotools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 38.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 36.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=390977 sha256=aedbe7756bd67d32fb35d19b7bedfbc78c0717915a1396d62eb42b2d0afedddc
  Stored in directory: /home/mkurtys/.cache/pip/wheels/89/5c/59/e3cd7600cf164a071163f9fed6550a29185d64345591f7d9fe
Successfully built pycocotools
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24

In [8]:
import glob
import cv2
import functools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pydicom
import pydicom.pixel_data_handlers
import numpy as np
import cv2
import os
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from pathlib import Path
import random
import multiprocessing as mp
import itertools
from typing import Optional, Iterable
import seaborn as sns
import pandas as pd

%matplotlib inline

In [9]:
if jobs_count is None:
    print(f"Using jobs  { mp.cpu_count()-1}")

Using jobs  3


In [10]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_LINEAR):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)

    return resized


In [11]:
 


def normalization_with_min_max(img, val_min, val_max):
    if val_min<val_max:
        return (img - val_min)/(val_max - val_min)
    else:
        return img

def normalization(img):
    val_min = np.amin(img)
    val_max = np.amax(img)
    return normalization_with_min_max(img, val_min, val_max), val_min, val_max
    

def truncation_normalization(img, val_min, val_max):
    truncated = np.clip(img,val_min, val_max)
    normalized = (truncated - val_min)/(val_max - val_min)
    return normalized

In [12]:

def find_contours(img, 
                  threshold_level: int):
    blured = cv2.GaussianBlur(img, (7, 7), 0)
    
    thresholded = np.zeros_like(blured, dtype=np.uint8)
    thresholded[ blured >= threshold_level ] = 1
    
    cnts, _ = cv2.findContours(thresholded,
                               cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_SIMPLE)
    # print(f" contours len {len(cnts)}")
    
    return cnts

def largest_countour_and_coords_or_image_size(cnts, img):
    if cnts:
        cnt = max(cnts, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        return cnt, (x,y,w,h)
    else:
        return None, (0, 0, img.shape[1], img.shape[0])



#  'WindowCenter',
#  'WindowWidth',
#  'RescaleIntercept',
#  'RescaleSlope',
#  'RescaleType',
#  'VOILUTFunction',

def data_to_monochrome2_if_needed(data, ds):     
    if ds.PhotometricInterpretation == "MONOCHROME1":
        data_max = np.amax(data)
        return data_max - data  
    else:
        return data

def img_crop(img, threshold_level):
    contours = find_contours(img, threshold_level=threshold_level)
    #cv.drawContours(	image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]	) -> 	image
    max_cnt, cnt_coords = largest_countour_and_coords_or_image_size(contours, img)
    max_cnt_mask=None
    max_cnt_mask = np.zeros_like(img, dtype=np.uint8)
    if max_cnt is not None:
        cv2.drawContours(max_cnt_mask, [max_cnt], -1 , 1, cv2.FILLED)
        croped_img = np.where(max_cnt_mask > 0 , img, max_cnt_mask)
        # img = cv2.bitwise_or(img, img, mask=max_cnt_mask)
        x,y,w,h = cnt_coords
        croped_img = croped_img[y:y+h, x:x+w]
    else:
        croped_img = img.copy()
        
    return croped_img, cnt_coords, max_cnt_mask 

def process(filename):
    ds = pydicom.dcmread(filename)
    data = ds.pixel_array
    voi_data = pydicom.pixel_data_handlers.apply_voi_lut(data, ds)
    # data = data_to_monochrome2_if_needed(data, ds)
    voi_data = data_to_monochrome2_if_needed(voi_data, ds)
    img_croped, crop_coords, _ = img_crop(voi_data, threshold_level=5)
    
    img_croped, _, _ = normalization(img_croped)
    img_croped = img_croped*U16MAX
    img_croped=img_croped.astype(np.uint16)
    return data, img_croped, crop_coords

def process_and_save(filename, output_dir, resize_longer_axis_to=None):
    original_img, img_croped, crop_coords = process(filename)
    
    if resize_longer_axis_to:
        img_croped=image_resize(img_croped, height=resize_longer_axis_to)
    
    png_filename = str(filename.stem + ".png")
    directory = Path(output_dir, *filename.parts[-3:-1])
    directory.mkdir(parents=True, exist_ok=True)
    path_to_save = directory/png_filename
    # print(path_to_save)
    cv2.imwrite(str(path_to_save), img_croped)
    return (original_img.shape[1], original_img.shape[0]), crop_coords

def process_and_save_star(args):
    return process_and_save(*args)

In [13]:
def bulk_process_and_save(input_directory,
                          resize_longer_axis_to=512,
                          limit_files=None,
                          files_border_percent=None,
                          files_border_process_lower=True,
                          jobs_count=None):
    jobs_count = mp.cpu_count()-1 if jobs_count is None else jobs_count

    
    output_path = Path(".")
    files = [image_path for directory_path in input_directory.iterdir()
             for image_path in directory_path.iterdir()]
    
    # if resize_dims:
    #    for resize_dim in resize_dims:
    #        os.makedirs( os.path.join(str(output_path), str(resize_dim)),
    #                    exist_ok=True)
    
    
    
    if limit_files:
        files = files[:limit_files]
        
    if files_border_percent:
        total_files_len = len(files)
        border_guess = int(len(files)*files_border_percent)
        # continue while all images for patient will be processed
        border = border_guess
        start_patient_id = files[border].parts[-2]
        patient_id = start_patient_id
        print(start_patient_id)
        while border < len(files) and patient_id==start_patient_id:
            border+=1
            patient_id = files[border].parts[-2]
            
            
        if files_border_process_lower:
            files = files[:border]
        else:
            files = files[border:]
        print(f"out of {total_files_len}, the {len(files)} will be processed")
        
    
    os.makedirs(str(output_path), exist_ok=True)
    # opencv use threads, it may be somewhat faster, but may not
    
    if jobs_count>0:
        with mp.Pool(jobs_count) as p:
            t = tqdm(p.imap(process_and_save_star,
                                  zip(files, 
                                      itertools.repeat(output_path),
                                      itertools.repeat(resize_longer_axis_to))),
                     total=len(files))
            dims =  list(t)
    else:
        dims = []
        for file in tqdm(files):
            d=process_and_save(file, output_path, resize_longer_axis_to)
            dims.append(d)
    dims_df = pd.DataFrame.from_records(dims, columns=["size", "crop"])
    dims_df["filepath"] = files
    return dims_df
            
    

In [16]:
crop_df= bulk_process_and_save(competition_path/"train_images",
                               resize_longer_axis_to=resize_dim,
                               limit_files=limit_files,
                               files_border_percent=files_border_percent,
                               files_border_process_lower=files_border_process_lower,
                               jobs_count=jobs_count)
crop_df.to_csv("crop_train.csv")

  0%|          | 0/54706 [00:00<?, ?it/s]

In [17]:
crop_df= bulk_process_and_save(competition_path/"test_images",
                               resize_longer_axis_to=resize_dim,
                               limit_files=None,
                               files_border_percent=None,
                               jobs_count=jobs_count)
crop_df.to_csv("crop_test.csv")

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# copy test train
!cp {competition_path}/test.csv .
!cp {competition_path}/train.csv .